# Agente Inteligente para la Generación Automática de Queries MongoDB a partir de Lenguaje Natural

Este notebook demuestra el uso de un agente inteligente capaz de transformar instrucciones en lenguaje natural en queries MongoDB, incluyendo soporte para joins dinámicos.

In [1]:
import json
with open('../datasets/transactions_collection.json') as f:
    data = json.load(f)
print(f'Total registros: {len(data)}')

Total registros: 8


In [ ]:
import sys
sys.path.append('..')
sys.path.append('../src')
from AgenteGeneradorQueryMongo import SmartMongoQueryGenerator

generator = SmartMongoQueryGenerator()

# Ejemplo de instrucción en lenguaje natural para JOIN (proyección dinámica de campos del join)
instruccion_join = "une la colección empleados con la colección departamentos usando el campo departamento_id proyecta los campos nombre, apellido, departamentos_info, departamento_nombre"
query_join = generator.generate_query('empleados', instruccion_join)
print('Query generada para join:')
print(query_join)

Query generada para join:
db.getCollection("empleados").aggregate([
  {
    $lookup: {
      from: "departamentos",
      localField: "departament_id",
      foreignField: "departament_id",
      as: "departamentos_info"
    }
  },
  {
    $unwind: "$departamentos_info"
  },
  {
    $project: {
      _id: 0,
      departamentos_info.departamento_nombre: "$departamentos_info.departamento_nombre",
      nombre: "$nombre",
      apellido: "$apellido"
    }
  }
])


Puedes modificar la variable `instruccion` para probar otras frases en lenguaje natural, incluyendo joins con diferentes nombres de colección y campo, o instrucciones que no requieran join.

# Ejemplos avanzados de instrucciones en lenguaje natural
A continuación se muestran tres casos de uso para probar el agente generador de queries MongoDB:

In [3]:
# Caso 1: Header simple
instruccion_header = """
crear campo dateMascara que convierta el campo Date a formato %Y%m%d usando los primeros 19 caracteres
crear campo reg que concatene: "1", "002", el campo Date convertido a formato %Y%m%d%H%M%S usando los primeros 19 caracteres, "00", "01", un espacio, un salto de línea, otro salto de línea
"""
print('--- Caso 1: Header ---')
query_header = generator.generate_query('header', instruccion_header)
print(query_header)

--- Caso 1: Header ---
db.getCollection("header").aggregate([
  {
    $project: {
      _id: 0,
      dateMascara: {
        $dateToString: {
          date: {
            $dateFromString: {
              dateString: {
                $substr: [
                  "$Date",
                  0,
                  19
                ]
              }
            }
          },
          format: "%Y%m%d"
        }
      },
      reg: {
        $concat: [
          "1",
          "002",
          {
            $dateToString: {
              date: {
                $dateFromString: {
                  dateString: {
                    $substr: [
                      "$Date",
                      0,
                      19
                    ]
                  }
                }
              },
              format: "%Y%m%d%H%M%S"
            }
          },
          "00",
          "01",
          " ",
          "
",
          "
"
        ]
      }
    }
  }
])


In [4]:
# Caso 2: Detalle avanzado
instruccion_detalle = """
desanidar Devices
desanidar Devices.ServicePoints
desanidar Devices.ServicePoints.ShipOutCycles
desanidar Devices.ServicePoints.ShipOutCycles.Transactions
agrupar por date, deviceId, branchCode, subChannelCode, shipOutCode, currencyCode, confirmationCode y sumar el total de Devices.ServicePoints.ShipOutCycles.Transactions.Total
proyectar los caracteres de la posición 2 en adelante del deviceId
crear campo confirmationCode que sea _id.confirmationCode o " " si es nulo
crear campo totalParteEntera que sea el primer elemento del split del total por punto
crear campo totalParteDecimal que sea el segundo elemento del split del total por punto o "00" si es nulo
crear campo reg que sea la concatenación de "5", la condición de moneda, la fecha, "00", el deviceId con padding, el shipOutCode con padding, la condición de sucursal, el monto con padding y el código de confirmación con padding
ordenar por deviceId y shipOutCode y subChannelCode y currencyCode
"""
print('--- Caso 2: Detalle ---')
query_detalle = generator.generate_query('detalle', instruccion_detalle)
print(query_detalle)

--- Caso 2: Detalle ---
db.getCollection("detalle").aggregate([
  {
    $unwind: "$Devices"
  },
  {
    $unwind: "$Devices.ServicePoints"
  },
  {
    $unwind: "$Devices.ServicePoints.ShipOutCycles"
  },
  {
    $unwind: "$Devices.ServicePoints.ShipOutCycles.Transactions"
  },
  {
    $group: {
      _id: {
        Date: "$Date",
        deviceId: "$Devices.Id",
        branchCode: "$Devices.BranchCode",
        subChannelCode: "$Devices.ServicePoints.ShipOutCycles.SubChannelCode",
        shipOutCode: "$Devices.ServicePoints.ShipOutCycles.Code",
        currencyCode: "$Devices.ServicePoints.ShipOutCycles.Transactions.CurrencyCode"
      },
      total: {
        $sum: "$total"
      }
    }
  },
  {
    $sort: {
      deviceId: 1,
      shipOutCode: 1,
      subChannelCode: 1
    }
  },
  {
    $project: {
      deviceId: {
        $substrCP: [
          "$_id.deviceId",
          2,
          {
            $strLenCP: "$_id.deviceId"
          }
        ]
      },
      reg: {
      

In [5]:
# Caso 3: Detalle con agrupaciones y padding
instruccion_detalle2 = """
desanidar Devices
desanidar Devices.ServicePoints
desanidar Devices.ServicePoints.ShipOutCycles con preserveNullAndEmptyArrays
desanidar Devices.ServicePoints.ShipOutCycles.Transactions con preserveNullAndEmptyArrays
agrupar por deviceId, branchCode, subChannelCode, shipOutCode, currencyCode y sumar el total de Devices.ServicePoints.ShipOutCycles.Transactions.Total en soles y en dólares según el código de moneda
luego agrupa todo y suma totalSoles y totalDolares, y cuenta total de registros en soles y en dólares según el código de moneda
crear campo totalParteEnteraSoles que sea el primer elemento del split de totalSoles por punto
crear campo totalParteDecimalSoles que sea el segundo elemento del split de totalSoles por punto o "00" si es nulo
crear campo totalParteEnteraDolares que sea el primer elemento del split de totalDolares por punto
crear campo totalParteDecimalDolares que sea el segundo elemento del split de totalDolares por punto o "00" si es nulo
crear campo reg que concatene: "9", el total de registros con padding, el total de registros en soles con padding, el total de registros en dólares con padding, el monto en soles con padding, el monto en dólares con padding, un salto de línea, otro salto de línea
"""
print('--- Caso 3: Detalle agrupado ---')
query_detalle2 = generator.generate_query('detalle2', instruccion_detalle2)
print(query_detalle2)

--- Caso 3: Detalle agrupado ---
db.getCollection("detalle2").aggregate([
  {
    $unwind: "$Devices"
  },
  {
    $unwind: "$Devices.ServicePoints"
  },
  {
    $unwind: {
      path: "$Devices.ServicePoints.ShipOutCycles",
      preserveNullAndEmptyArrays: True
    }
  },
  {
    $unwind: {
      path: "$Devices.ServicePoints.ShipOutCycles.Transactions",
      preserveNullAndEmptyArrays: True
    }
  },
  {
    $group: {
      _id: {
        deviceId: "$Devices.Id",
        branchCode: "$Devices.BranchCode",
        subChannelCode: "$Devices.ServicePoints.ShipOutCycles.SubChannelCode",
        shipOutCode: "$Devices.ServicePoints.ShipOutCycles.Code",
        currencyCode: "$Devices.ServicePoints.ShipOutCycles.Transactions.CurrencyCode"
      },
      totalSoles: {
        $sum: {
          $cond: [
            {
              $eq: [
                "$Devices.ServicePoints.ShipOutCycles.Transactions.CurrencyCode",
                "PEN"
              ]
            },
            "$Devi